<a href="https://colab.research.google.com/github/theekshaya4/Image-colorization/blob/main/Copy_of_TASK_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import torch
from torchvision.models.segmentation import deeplabv3_resnet50, DeepLabV3_ResNet50_Weights
from torchvision import transforms
from PIL import Image
import numpy as np
import cv2
from google.colab import files

# ====== UPLOAD IMAGE ======
uploaded = files.upload()   # choose your image here (example: test.jpg)

# Get uploaded filename automatically
image_path = list(uploaded.keys())[0]

# ====== DEVICE ======
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

# ====== LOAD MODEL ======
weights = DeepLabV3_ResNet50_Weights.DEFAULT
model = deeplabv3_resnet50(weights=weights).eval().to(device)

# ====== LOAD IMAGE ======
img = Image.open(image_path).convert("RGB")

# ====== PREPROCESS ======
transform = transforms.Compose([
    transforms.ToTensor(),
])
tensor = transform(img).unsqueeze(0).to(device)

# ====== PREDICT ======
with torch.no_grad():
    out = model(tensor)["out"][0]

mask = out.argmax(0).cpu().numpy()

# ====== APPLY GREEN COLOR TO PERSON CLASS (15) ======
img_np = np.array(img)
img_np[mask == 15] = [0, 255, 0]

# ====== SAVE RESULT ======
cv2.imwrite("segmentation.jpg", cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR))

print("Saved as segmentation.jpg")